In [ ]:
import os, os.path
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
import random
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.keras.backend.floatx()
from collections import deque
import itertools
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from libs.utils import *
from libs.generate_boxes import *
from libs.dqn import *

In [ ]:
num_episode = 10000
global_step = 0
tr_l, h_fill, tr_r,avg_loss_l,history_eps,used_boxes_eps  = [],[],[],[],[],[]
K = 1
n_candidates = 5
num_max_remain = 180 
num_bbox_type = 20 #사용할 대박스 수
sc_target = 64 #최대 대박스 크기를 sc_target 크기만큼 줄임

In [ ]:
mi = 'U1'
mbox_type = 'PP'
f_name = 'plan_predict_input/' + mi+'_'+mbox_type
output_col = ['pack_no','creat_dd', 'odr_no', 'ctmmny_cd','vctp_cd', 'prod_no', 'mbox_cd', 'bbox_cd',
              'B_CBM',  'mbox_acctc_qty', 'bbox_acctc_qty', 
              'packing_rate_volume', 'packing_rate_cbm','x','y','z']

In [ ]:
# 대박스 규격 로드
df_bbox = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
casemaster=pd.read_csv('data/preprocessed/casemaster.csv')
df_bbox = df_bbox.iloc[:20]
df_bbox = pd.merge(df_bbox, casemaster[['CD', 'CBM']], how='left', left_on=['CASE_CD'], right_on=['CD']).drop(['CD'],axis=1)
df_bbox = df_bbox.drop(['CASE_CD_count'], axis=1)

In [ ]:
# 대박스 규격 로드 (Env)
bbox_size = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
bbox_size = bbox_size.values[:num_bbox_type, 1:-1]
bbox_size = np.concatenate([np.sort(bbox_size[:,:2], axis=1), bbox_size[:,2:]], axis=-1)

In [ ]:
# 대박스 스케일링
sc = sc_target/np.max(bbox_size, axis =0)[:2] 
bbox_sc = bbox_size.copy() 
bbox_sc[:, :2] = bbox_size[:,:2] * sc # 세로가로만 스케일링
bbox_sc = (bbox_sc+0.5).astype('int') #반올림
mxl, mxb, mxh = np.max(bbox_sc, axis =0 ) # 최대 규격
print(mxl, mxb, mxh)

In [ ]:
# 대박스 행렬로 변환
bbox_type = [] #np.zeros((1, 20, 20))
for l,b,h in bbox_sc:
    bbox = np.zeros((mxl, mxb))
    bbox[l:,:] = mxh
    bbox[:,b:] = mxh
    bbox[:l,:b] = mxh - h
    bbox_type.append(bbox)
bbox_type = np.stack((bbox_type)) #(5, 20, 20)

In [ ]:
# 중박스 데이터 로드 
group_key = ['creat_dd', 'odr_no', 'ctmmny_cd', 'mbox_type']
df = pd.read_csv('data/preprocessed/'+f_name +'_resid.csv')
org_col = df.columns
df = df.sort_values('sum_wt', ascending = False)
df = df.set_index(group_key).sort_index()
group_idx = df.index.drop_duplicates()

In [ ]:
# 중박스 데이터 로드 (입력 데이터)
with open('data/preprocessed/' + f_name +'_resid.pickle', 'rb') as handle: 
    boxes_multi =  pickle.load(handle)
with open('data/preprocessed/' + f_name +'_resid_v0.pickle', 'rb') as handle: 
    boxes_multi_n =  pickle.load(handle)

boxes_multi = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi]
boxes_multi_n = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi_n]

In [ ]:
# 중박스 스케일링
boxes_multi_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, 2:3]], axis=-1).astype('int') for x in  boxes_multi]
boxes_multi_n_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, [2,5]]], axis=-1).astype('int') for x in  boxes_multi_n]
cbm_per_mbox = [x[:,-1] for x in boxes_multi]

In [ ]:
# 중박스 크기를 행렬로 표현
max_size = sc_target
boxes_mat_multi = np.stack([size2mat(x, num_max_remain, max_size) for x in boxes_multi_n_sc]) # (N, num_max_remain, max_size, 2)

In [ ]:
# Environment
env = Bpp3DEnvMS(length = mxl, breadth = mxb, height = mxh, bbox_type = bbox_type)
# Agent
agent = DQNAgent( L = mxl, B = mxb , H = mxh, n_remains = num_max_remain, n_loading=K, max_size = max_size,lr=1e-4, exp_steps=9000, 
                 train_st = 1500, memory_len=2000, update_target_rate = 1000, net = 'DDQN_DNN')

In [ ]:
is_pred = False

if is_pred:
    num_episode = len(boxes_multi_sc)
    # 예측 결과 테이블
    pk = pd.DataFrame(columns = ['pack_no']+list(org_col)+['packing_rate_cbm','packing_rate_volume','x','y','z'])
    pack_no = 0
    agent.epsilon = 0

In [ ]:
for e in range(num_episode):
    st = time.time()
    step = 0
    
    if is_pred:
        df_p = df.loc[group_idx[e]].reset_index().copy()
        df_p = df_p.loc[df_p.index.repeat(df_p['num_mbox'])]
        
    else:
        if e > 9000:
            agent.epsilon = 0
        if agent.epsilon > agent.epsilon_end and len(agent.memory) > = agent.train_start:
            agent.epsilon -= agent.epsilon_decay_step
            
    ith_data = e % len(boxes_multi)#5#
    boxes_all_sc = boxes_multi_sc[ ith_data].copy().astype('int')# 
    cbm_all = cbm_per_mbox[ith_data].copy()
    r_boxes = boxes_all_sc.copy()
    r_mat = boxes_mat_multi[ith_data].copy() #(50,64,2)
    boxes_idx = np.arange(len(boxes_all_sc))
    loaded_idx = []
    used_boxes, pred_pos = [],[]
    env_rewards, is_last, used_bbox = [], [], []
    history, h_load, h_remain_size, h_load_size = [],[],[],[]
    t_history, t_load, t_remain_size, t_load_size = [],[],[],[]
    
    while len(r_boxes) > 0:
        #done = False
        len_group = 0
        cbm_reward = 0
        
        # state 선택
        in_state = np.array( [ np.concatenate([np.zeros((env.length,env.breadth,2)), b[...,np.newaxis]], axis=-1) for b in bbox_type] )#(5,20,20,3)
        in_state = (in_state/env.height).astype(np.float32)
        loading_loc_c = np.zeros((num_bbox_type, env.length, env.breadth, K)) # 적재 위치
        in_loading = np.zeros((num_bbox_type, K, max_size, 2))
        in_remain = np.array([r_mat] * num_bbox_type).astype(np.float32)
        
        k = min(K, len(r_boxes))
        selected, selected_idx = cbn_select_boxes(r_boxes[:n_candidates], boxes_idx[:n_candidates], k)
        s_order, s_order_idx = get_selected_order(selected, selected_idx, k)
        s_order, s_order_idx = rot_one_order(s_order, s_order_idx)
        
        bbox_idx = bbox_agent.get_action() #r_boxes, selected?s_order?
        #env.reset(bbox_idx)
        history.append(in_state[bbox_idx])
        h_load.append(loading_loc_c[bbox_idx])
        h_remain_size.append(in_remain[bbox_idx])
        h_load_size.append(in_loading[bbox_idx])
        
        state = env.container_s.copy()
        state_h = env.container_h.copy()
        global_step += 1
        step += 1        
        cbm_all_ratio = cbm_all / df_bbox.iloc[env.bbox_idx]['CBM']
        
        order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                get_selected_location(s_order, s_order_idx, state, state_h, env.height,K, cbm_reward ,cbm_all_ratio ) #위치